In [ ]:
# import necessary packages
import numpy as np
import pandas as pd

# path to data we just downloaded
# if you are interested in how modelpoints were created,
# check out the `generating_modelpoints.ipynb` file in the repo
DATA_XLS = "./dstoolkit/model_point.xlsx"

# ingesting our data into a dataframe (df)

### Creating & editting spreadsheets from CLI with `openpyxl` & 'xlwings`

In [6]:
%pip install openpyxl appscript psutil xlwings

Note: you may need to restart the kernel to use updated packages.


In [4]:
import xlwings as xw
import pandas as pd

# Interactive mode: open new workbook
# write pd.DataFrame to "A1" then read it back

book = xw.Book()
#sheet = book.sheets[0]
#sheet.range("A1").value = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})


In [9]:
list(dir(sheet))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'activate',
 'api',
 'autofit',
 'book',
 'cells',
 'charts',
 'clear',
 'clear_contents',
 'clear_formats',
 'copy',
 'delete',
 'impl',
 'index',
 'name',
 'names',
 'page_setup',
 'pictures',
 'range',
 'render_template',
 'select',
 'shapes',
 'tables',
 'to_html',
 'to_pdf',
 'used_range',
 'visible']